In [2]:
import pickle
import pandas as pd
import transformers
from src.utils import *

https://api-inference.huggingface.co/models/s-nlp/roberta-base-formality-ranker


In [10]:
dialogue_df = pd.read_csv('/Users/madisonthantu/Desktop/COMS_6998/Final_Project/recursive_LLMs/Data/initial_datasets/dialogue/full_data.csv')

In [11]:
df_train = dialogue_df.iloc[:200]
df_validate = dialogue_df.iloc[200:250]

In [14]:
df_train.to_csv('Data/dialogue_train_small.csv', index=False)
df_validate.to_csv('Data/dialogue_validation_small.csv')

In [1]:
import os 
import inspect 
print(os.path.dirname(inspect.getfile(inspect))+"/site-packages") 

/Users/madisonthantu/miniforge3/lib/python3.10/site-packages


In [8]:
pd.read_csv('/Users/madisonthantu/Desktop/COMS_6998/Final_Project/recursive_LLMs/Data/Data/initial_datasets/dialogue/validation_data.csv')#.to_csv('/Users/madisonthantu/Desktop/COMS_6998/Final_Project/recursive_LLMs/Data/Data/initial_datasets/dialogue/training_data.csv')

,id,document,summary
0,13811546,Caroline: I don't feel like going out tonight\...,Caroline doesn't feel like going out tonight. ...
1,13729119,Alaine: Hey\r\nAlaine: Are we still having our...,Alex and Alaine are having a date tomorrow. Al...
2,13729613,Arabella: Why is this Jada Smith person all ov...,The actress Jada Smith overshares her personal...
3,13813146,Victoria: Hi! :) How are you? Haven't heard fr...,Connor is learning to code in Java and Python....
4,13729655,"Frank: Hey, Vicky\r\nFrank: I have to ask you:...",Vicky was criticized by her mother during fami...
...,...,...,...
4906,13729067,Annabelle: I’m sorry for your loss.\r\nKian: T...,Kian can talk to Annabelle if he needs it.
4907,13729334,"Julius: dude, your assessment of manutd\r\nLaw...",Lawrence is disappointed with Manchester Unite...
4908,13864953,Ben: Guess what! :D\nDavid: What? :D\nBen: Jus...,Ben just got a new pad.
4909,13716163,Maya: Are you ready?\r\nNatalie: need 10 more ...,Maya is going to wait for Natalie and Richard ...


In [9]:
pd.read_csv('/Users/madisonthantu/Desktop/COMS_6998/Final_Project/recursive_LLMs/Data/Data/initial_datasets/dialogue/validation_data.csv').to_csv('/Users/madisonthantu/Desktop/COMS_6998/Final_Project/recursive_LLMs/Data/Data/initial_datasets/dialogue/validation_data.csv', index=False)
pd.read_csv('/Users/madisonthantu/Desktop/COMS_6998/Final_Project/recursive_LLMs/Data/Data/initial_datasets/dialogue/validation_data.csv')

,id,document,summary
0,13811546,Caroline: I don't feel like going out tonight\...,Caroline doesn't feel like going out tonight. ...
1,13729119,Alaine: Hey\r\nAlaine: Are we still having our...,Alex and Alaine are having a date tomorrow. Al...
2,13729613,Arabella: Why is this Jada Smith person all ov...,The actress Jada Smith overshares her personal...
3,13813146,Victoria: Hi! :) How are you? Haven't heard fr...,Connor is learning to code in Java and Python....
4,13729655,"Frank: Hey, Vicky\r\nFrank: I have to ask you:...",Vicky was criticized by her mother during fami...
...,...,...,...
4906,13729067,Annabelle: I’m sorry for your loss.\r\nKian: T...,Kian can talk to Annabelle if he needs it.
4907,13729334,"Julius: dude, your assessment of manutd\r\nLaw...",Lawrence is disappointed with Manchester Unite...
4908,13864953,Ben: Guess what! :D\nDavid: What? :D\nBen: Jus...,Ben just got a new pad.
4909,13716163,Maya: Are you ready?\r\nNatalie: need 10 more ...,Maya is going to wait for Natalie and Richard ...


In [4]:
pd.read_csv('Data/dialogue_validation.csv')

,Unnamed: 0.1,Unnamed: 0,id,document,summary
0,0,200,13820762,Rachel: HAPPY NEW YEAAAAAAR!! ヽ(^o^)丿\r\nRache...,'i see Rachel: wish you were here': 'i see you...
1,1,201,13730176,Bobby: Did you see the episode of I'm a Celeb ...,"i'm a celeb episode was super gross. worms, te..."
2,2,202,13829860,Daniel: hi! any free slots this week?\r\nDavid...,i'll be at work this week. i'll let you know.
3,3,203,13715771,Frederic: Mornin'! How's Monday? Mine sucks.\r...,"Alma: Hey, hey! I hate Mondays!"
4,4,204,13820057,Ron: hey dad :)\r\nKurt: how much do you need ...,Ron: i wanted to be nice... 50 dollars. Kurt: ...
5,5,205,13862422,Ali: <file_photo>\nLucie: Nice colour ;)\nAli:...,Ali: Toothpaste is mine Lucie: I was afraid th...
6,6,206,13828253,"Nicole: Soph, could we meet a little later tom...",Sophia: I got invited to a job interview. Nico...
7,7,207,13829964,"Marilyn: Mark, darling, are you free to talk n...","Marilyn: ok, darling, are you free to talk now..."
8,8,208,13680188,Julia: karaoke on friday night?\r\nAva: call!\...,"Julia: maybe someone else will come Ava: ok, I..."
9,9,209,13716428,Tom: Did anyone pick up prints?\r\nGlen: I did...,"Glen: I didn't pick up prints, but called John..."
